Importing necessary modules

In [ ]:
from edempy import Deck
from pyfedic.mesh import Mesh, gen_mesh
from pyfedic.cells import T4
import numpy as np
from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation as R
from pathlib import Path
import pickle

Functions definition

In [ ]:
if True:
    # "Optional : to see some debug stuff"
    import logging
    from logging import FileHandler
    from pyfedic.tictoc import TICTOC

    logging.basicConfig(
        level=logging.INFO,
        format="%(levelname)s :: %(message)s"
    )
    logging.getLogger().setLevel(TICTOC)
    

# Function to extract required data from EDEM file
def dataT(deck,t):
    """
    Parameters
    ----------
    deck : deck
    t : Timestep
    Returns
    -------
    dataS : [ids , Xpos, Ypos, Zpos, Radius, Quaternions(w,x,y,z)]
    """
    print(f"________Preparing data for the time step {t} ________")
    pids = []
    ppos = []
    p_radUS = []
    QuaternionsUS = []
    
    for i in range(deck.timestep[t].numTypes):
        #condition to check if particles of this type are created
        if deck.timestep[t].particle[i].getNumParticles() > 0: 
            p = deck.timestep[t].particle[i].getIds()
            pos = deck.timestep[t].particle[i].getPositions()
            rad = deck.timestep[t].particle[i].getSphereRadii()
            quaternions = deck.timestep[t].particle[i].getOrientation() # euler axis of rotation of particles
            
            pids.extend(p)
            ppos.extend(pos)
            p_radUS.extend(rad)
            QuaternionsUS.extend(quaternions)

    pids = np.array(pids)
    ppos = np.array(ppos)
    p_radUS = np.array(p_radUS)
    QuaternionsUS = np.array(QuaternionsUS)
    
    dataUS = np.column_stack((pids,ppos,p_radUS,QuaternionsUS)) #combined data in the array format and is unsorted
    dataS = dataUS[dataUS[:,0].argsort()] # Sorted data
    
    return dataS


# Function for correcting displacements along the periodic boundaries
def DispAlongPeriodicBounds(data_i, data_f, X_bounds = [-25.5,25.5], Y_bounds = [-25.5,25.5]):
    """
    Parameters
    ----------
    data_i : Array [X Y Z]
    data_f : Array [X Y Z]
    for the initial and final arrays the X and Y data should be in the 2nd and 3rd columns

    X_bounds : list [X_min, X_max], optional
        The default is [-25.5,25.5].
    Y_bounds : list [Y_min, Y_max], optional
        The default is [-25.5,25.5].

    Returns
    -------
    Displacements : [ Ux Uy Uz ]

    """
    Xbl, = np.diff(X_bounds)
    Ybl, = np.diff(Y_bounds)

    Ux, Uy, Uz = (data_f - data_i).T

    out = Ux > Xbl/2
    Ux[out] -= Xbl
    out = Ux < -Xbl/2
    Ux[out] += Xbl

    out = Uy > Ybl/2
    Uy[out] -= Ybl
    out = Uy < -Ybl/2
    Uy[out] += Ybl

    return np.vstack((Ux, Uy, Uz)).T

File opening and settings

In [1]:
file_name = r"FileName.dem"

In [ ]:
# Settings for performing Homogenization

# include rotations for the calculations
Include_Rotations = True
#Save particle density data
ParticleDensityOutput = False
# perform for onlu one timestep:
one_Time = True

# Size of element for meshing and homogenization
Element_size = 2

# 50x50
# X_mesh = [-20, 20]
# Y_mesh = [-20, 20]

# 100x100
# X_mesh = [-40, 40]
# Y_mesh = [-40, 40]

# 200x200
X_mesh = [-90, 90]
Y_mesh = [-90, 90]

Z_mesh = [-38, 42]
# Z_mesh = [-37, 38]

Check if the file is already saved and open if necessary

In [ ]:
# Extract deck from EDEM file

file_path = Path(file_name)
pickle_Save = file_path.parent
pickleFileName = file_path.parts[-1][0:-4] +'.pickle'

try:
    saveDeck = False
    with open( pickle_Save / pickleFileName , 'rb') as file:
        deck = pickle.load(file)
    print("________Pickle file available")
    
    #  check if the file is in proper location
    if(deck.__dict__['deckname'] != file_name):
        print('Old pickle file needs update')
        saveDeck = True

except FileNotFoundError:
    print("________Pickle file not found")
    saveDeck = True

if saveDeck == True:
    print("________Extracting deck")
    deck = Deck(file_name) # opening the deck
    print("________Extracting deck : done")
    print("________Creating a Pickle file")

    with open( pickle_Save / pickleFileName, 'wb') as file:
        pickle.dump(deck, file) # saving as pickle file as it is easier to load

In [ ]:
# Time start End and step size
t_b = 5 #corresponds to timestep 3500
t_final = len(deck.timestepKeys)-1
step_size = 20

# Periodic boundary limits
X_Lims = [deck.timestep[0].domainMin[0] , deck.timestep[0].domainMax[0]]
Y_Lims = [deck.timestep[0].domainMin[1] , deck.timestep[0].domainMax[1]]

In [ ]:
# establish the saving folder
save_folder = Path(file_name).parent
Sub_folder = 'Output_Strain_IncludingRotation_E' + str(Element_size)
(save_folder / Sub_folder).mkdir(parents=True, exist_ok=True)
(save_folder / Sub_folder / 'Results_Delaunay').mkdir(parents=True, exist_ok=True)
(save_folder / Sub_folder / 'Results_interpolated_grid').mkdir(parents=True, exist_ok=True)

# Timesteps required
t_steps = np.arange(t_b+5,t_final,step_size).tolist()
if (t_steps[-1] != t_final): t_steps.append(t_final)
t_steps.insert(0,t_b)

In [ ]:
if Include_Rotations == True:
    
    (save_folder / Sub_folder / 'ResultsWithRotation').mkdir(parents=True, exist_ok=True)
    (save_folder / Sub_folder / 'ResultsWithRotation' / 'Results_Delaunay').mkdir(parents=True, exist_ok=True)
    (save_folder / Sub_folder / 'ResultsWithRotation' / 'Results_interpolated_grid').mkdir(parents=True, exist_ok=True)
    
    data0 = dataT(deck, t_b) # [ids , Xpos, Ypos, Zpos, Radius, Quaternions(w,x,y,z)]
    Quaternions0 = data0[:,5::]
           
    # initial vector representing the orientation of the moment of inertia axis
    # intersecting two points on the sphere at [0,0,1] and [0,0,-1]
    Num_nodes = len(data0)*2 # two points for each particle
    Unit_Moi_Vec = np.zeros((Num_nodes, 3))
    Unit_Moi_Vec[0::2] = 1
    Unit_Moi_Vec[1::2] = -1
    
    Node_Quats = np.zeros((Num_nodes, 4))
    Node_Quats[0::2] = Quaternions0
    Node_Quats[1::2] = Quaternions0
    
    Rotated_unit_Vec = R.from_quat(Node_Quats).apply(Unit_Moi_Vec)
    
    nodes0 = np.zeros((Num_nodes, 3))
    nodes0[0::2] = data0[:,1:4] + data0[:,4].reshape(-1,1) * Rotated_unit_Vec[0::2]
    nodes0[1::2] = data0[:,1:4] + data0[:,4].reshape(-1,1) * Rotated_unit_Vec[1::2]
    
    # Apply tessellation 
    cells = Delaunay(nodes0).simplices
    m = Mesh.new(nodes0, {T4: cells})
    nm = gen_mesh(
        (X_mesh[0], X_mesh[1]),
        (Y_mesh[0], Y_mesh[1]),
        (Z_mesh[0], Z_mesh[1]),
        elt_size = Element_size
    )
        
    for i, t in enumerate(t_steps):
        print('step', i)
        
        data = dataT(deck, t)
        Quats = data[:,5::]
              
        Node_Quats = np.zeros((Num_nodes, 4))
        Node_Quats[0::2] = Quats
        Node_Quats[1::2] = Quats
        
        Rotated_unit_Vec = R.from_quat(Node_Quats).apply(Unit_Moi_Vec)
        
        nodes = np.zeros((Num_nodes, 3))
        nodes[0::2] = data[:,1:4] + data[:,4].reshape(-1,1) * Rotated_unit_Vec[0::2]
        nodes[1::2] = data[:,1:4] + data[:,4].reshape(-1,1) * Rotated_unit_Vec[1::2]

        #correct the displacement across periodic boundaries
        U = DispAlongPeriodicBounds(nodes0, nodes, X_bounds = [-25.5,25.5], Y_bounds = [-25.5,25.5])

        # m.save(save_folder / Sub_folder / 'ResultsWithRotation'/ 'Results_Delaunay' / f'Result_{t:04d}.vtk', U, compute_eps=True)
        nU = m.interp_V(U, nm, out=np.nan)
        nU = nm.mean_V(nU, 2)
        nm.save(save_folder / Sub_folder / 'ResultsWithRotation'/ 'Results_interpolated_grid' / f'Result_N{t:04d}.vtk', nU, compute_eps=True)
        
        Unit_Moi_Vec = np.copy(Rotated_unit_Vec)

In [ ]:
#Triangulation for the initial timestep
Data_B = dataT(deck, t_b)
nodes = Data_B[:,1:4]
cells = Delaunay(nodes).simplices
m = Mesh.new(nodes, {T4: cells})
nm = gen_mesh(
    (X_mesh[0], X_mesh[1]),
    (Y_mesh[0], Y_mesh[1]),
    (Z_mesh[0], Z_mesh[1]),
    elt_size = Element_size
)

for i, t in enumerate(t_steps):
    print('step', i)
    data = dataT(deck, t)[:,1:4]

    #correct the displacement across periodic boundaries
    U = DispAlongPeriodicBounds(nodes, data, X_bounds = [X_Lims[0], X_Lims[1]], Y_bounds = [Y_Lims[0],Y_Lims[0]])

    # m.save(save_folder / Sub_folder / 'Results_Delaunay' / f'Result_{t:04d}.vtk', U, compute_eps=True)
    nU = m.interp_V(U, nm, out=np.nan)
    nU = nm.mean_V(nU, 2)
    nm.save(save_folder / Sub_folder / 'Results_interpolated_grid' / f'Result_N{t:04d}.vtk', nU, compute_eps=True)

In [ ]:
if ParticleDensityOutput == True:
    from pyfedic.io import write_mesh
    
    # The value of density is normalised to -1 for graphite particles and +1 for silicon particles
    density  = np.array([[1.99,-1],[2.8,-1],[3.6,-1],[4.8,-1],[6,-1],[6.5,-1],[7.5,-1],[0.61,1],[0.81,1],[1.02,1],[1.33,1],[1.63,1],[1.84,1],[2.04,1]])
    
    # Perform lookup based on the particle radii data
    P_density = np.zeros(len(Data_B))
    for i,j in enumerate(density[:,0]):
        P_density[Data_B[:,4] == j] = density[i,1]
    
    # generate new mesh for density
    m_d = Mesh.new(nodes, {T4: cells})
    
    nm_d = gen_mesh(
            (X_mesh[0], X_mesh[1]),
            (Y_mesh[0], Y_mesh[1]),
            (Z_mesh[0], Z_mesh[1]),
            zoi = Element_size
            )
    
    write_mesh(save_folder / Sub_folder / 'Density.vtk', m_d, point_values = {'Density': P_density.flatten()})
    
    nP_density = m_d.interp_V(P_density, nm_d, out=np.nan)
    nP_density = nm_d.mean_V(nP_density, 2)
    
    write_mesh(save_folder / Sub_folder / 'InterpolatedDensity.vtk', nm_d, point_values = {'Density': nP_density.flatten()})